# Implementación del método de Naive para identificar objetos en movimiento

**Índice**   
1. [Registro de frame](#id1)
2. [Funciones para implementación](#id2)
3. [Pruebas por frame](#id3)
4. [Segmentación de objetos en movimiento](#id4)
5. [Comparativa con métodos de mezclas gaussianas](#id5)
   


In [1]:
# Librerías principales
import numpy as np
import cv2 as cv
import matplotlib
from matplotlib import pyplot as plt

# Tipo de visualización
%matplotlib inline

# Versiones de librerías
print("".join(f"{x[0]}: {x[1]}\n" for x in [
    ("Numpy",np.__version__),
    ("openCV",cv.__version__),
    # ("Matplotlib",matplotlib.__version__),
]))

Numpy: 1.22.3
openCV: 4.5.5



In [2]:
# Definición de la ruta para levantar los videos
VD_DIR = r'.\videos'
VD_NAME = 'vtest.avi'


## Registro de frames<a name="id1"></a>
Se levanta el video que se empleará como test.


In [3]:
# Apertura de archivo y creación del objeto "capture"
#-------------------
capture = cv.VideoCapture(os.path.join(VD_DIR, VD_NAME))

if not capture.isOpened:
    print('Falla al abrir el archivo: ' + VD_NAME)
    exit(0)

Cada frame se guarda en un array. Se tratará como un tensor. La primera dimensión da cuenta de la cantidad de frame que posee el video y fueron guardados de esta forma. Luego se extraen valores a emplear luego.|

In [4]:
# Guardad de cada frame en array
frames = []
while True:
    read, frame= capture.read()
    if not read:
        break
    frames.append(frame)
frames = np.array(frames)

# Frames por segundo
fps = capture.get(cv.CAP_PROP_FPS)
# Cantidad de frames guardados como array
cant_frames = len(frames)

# Verificación de lectura correcta
if int(capture.get(cv.CAP_PROP_FRAME_COUNT)) == len(frames):
    print("La cantidad de frames guardados es correcta")
    print("Cantidad de frames registrados: ", cant_frames)
    print("Verificación de dimensiones: ", frames.shape)
else:
    print("No se ha guardado correctamente la cantidad de frames")



La cantidad de frames guardados es correcta
Cantidad de frames registrados:  795
Verificación de dimensiones:  (795, 576, 768, 3)


## Funciones para implementación<a name="id2"></a>
La implementación se separó en dos partes. La primera es la construcción del background. Se calcula como la mediana de una muestra aleatoria de frames. El cálculo es por canal y por pixel. Luego se arma la máscara o _foreground_. Para ello se toma el frame que se está leyendo en el momento de la reproducción del video. Se la mejora aplicando en pasos:
- Binarización tipo Otsu.
- Filtro morfológico de cierre, ya que conviene cerrar la figura para que no haya agujeros (ceros) dentro de los objetos que afecten la transformada.


In [5]:
def background_naive(input_frames, batch_size, seed, m_type='float32'):
    '''
    Función que devuelve la mediana, por canal, de un batch random formado por los frames de un video
    Los valores son de tipo flotante por defecto.
    - input_frames: array que contiene los frames de video a procesar.
    - batch_size: tamaño del batch que contiene las muestras seleccionadas aleatoriamente sin reemplazo.
    - seed: semilla para garantizar la repetitividad del proceso.
    '''
    rng = np.random.default_rng(seed)
    # Cantidad de frames a procesar
    cant_frames = len(input_frames)
    # Generación de enteros random
    idx = rng.choice(cant_frames, size=batch_size, replace=False, shuffle=False)
    # Armado del batch. Se convierte a tipo flotante
    batch = input_frames[idx,...].astype(m_type)

    # Mediana del batch
    return np.median(batch, axis=0)

In [6]:
def foreground_naive(background, input_frame, thresh_value=0, max_val=255):
    '''
    Función que devuelve la máscara del objeto que se encuentra en movimiento. Incluye la binarizacion, empleando el método Otsu,
    y la aplicación del filtro morfológico tipo cierre.
    - background: tensor de 3 canales con la mediana del batch aleatorio de frames
    - frame: frames de 3 canales con objetos a detectar.
    - tresh_value: valor umbral del método OTSHU.
    - max_val: valor máximo del método OTSHU.
    '''
    # Conversión del frame a float32
    input_frame.astype('float32')
    # resta
    diff = background - input_frame
    # normalizado
    diff = cv.normalize(diff,None,0,255,cv.NORM_MINMAX).astype('uint8')
    # pasaje a escala de grises
    diff_gray = cv.cvtColor(diff,cv.COLOR_BGR2GRAY)
    # binarización otsu
    ret, thresh = cv.threshold(diff_gray,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
    # Creamos un elemento estructurante y aplicamos operaciones morfologicas
    kernel = np.ones((3,3),np.uint8)
    # Filtro morfológico tipo cierre.
    closing = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel, iterations = 3)

    return closing
     

## Pruebas por frame<a name="id3"></a>

In [7]:
# # Selección de un frame del video: cambiar el número del primer índice
# f_p = frames[100,...]
# f_p.shape

In [8]:
# # background
# background_p = background_naive(frames, 50, 10)
# # foreground
# mask = foreground_naive(background_p, f_p)

# plt.imshow(mask, cmap='gray')
# plt.show()

In [9]:
# # Implementación de máscara
# indices = np.where(mask==255)
# f_p[indices[0], indices[1], :] = [200, 0, 255]
# plt.imshow(f_p)
# plt.show()

## Segmentación de objetos en movimiento<a name="id4"></a>
A partir de las funciones presentadas, se segmentó el video, de acuerdo a la capacidad del algoritmo de detectar los objetos en movimiento. En este caso particular se trató de personas. Los parámetros a definir previamente son:
- El tamaño del batch de frames.
- El tiempo de actualización del background.
- El valor de la semilla para garantizar la repetitibilidad de la prueba. 
  
Con los valores predefinidos, la implementación muestra buenos resultados de segmentación. Solo si se debe mencionar que no logra incluir la mayoría de los rostros en la segmentación. Ocurre también que, dependiendo el color de ropa, tampoco segmenta a la vestimenta en particular. Sin embargo son casos marginales y la figura principal en movimiento es detectada por completo. 

En cuanto a performance al correr el algoritmo, presenta el inconveniente de la actualización del background. La reproducción se detiene hasta tanto no finalice dicho armado. 

![imagen 1](./images_result/frame_seg_1.png) ![imagen 2](./images_result/frame_seg_2.png)


In [10]:
# Parámetros a definir para la segmentación
# -----------------------------------------
batch_size = 50 # tamaño del batch de frames para armar el background
t = 10.0 # tiempo de muestreo
seed = 122 # semilla para garantizar la repetitibilidad de la prueba


In [12]:
# Implementación de la segmentación de objetos en movimiento por Naive
# --------------------------------------------------------------------

# Definición de las posiciones de frame donde se realiza la actualización del background
update = np.arange(0, cant_frames, int(t*fps))
update = np.append(update,1) # se agrega un 1 al final para que sean coherentes las dimensiones, no afecta a la implementación

# Generación de semillas para garantizar la repetitibilidad de background generados aleatoriamente
rng = np.random.default_rng(seed)
seed_bg = rng.choice(4*len(update), size=len(update), replace=False, shuffle=False) 

# Contadores
count=0
s=0

# Apertura de archivo y creación del objeto "capture"
cap = cv.VideoCapture(os.path.join(VD_DIR, VD_NAME))
if not cap.isOpened:
    print('Falla al abrir el archivo: ' + VD_NAME)
    exit(0)

# Ciclo de segmentación
while True:
    r, f = cap.read()
    if not r:
        break
    
    # Actualización del background
    if count == update[s]:
        background = background_naive(frames, batch_size, seed_bg[s])
        s+=1
    # Contrucción del foreground
    foreground = foreground_naive(background, f)
    # Aplicación de la máscara al frame que se está leyendo
    idx_fg = np.where(foreground==255)
    f[idx_fg[0], idx_fg[1], :] = [200, 0, 255]
    # Salida
    cv.imshow('Foreground', foreground)
    cv.imshow('Video segmentado', f)

    count+=1

    # Se corre el video hasta que termine o se apriete escape
    keyboard = cv.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break
    elif keyboard == ord('s'):
        cv.imwrite('frame_seg.png',f)

cv.destroyAllWindows()
capture.release()

## Comparativa con métodos de mezclas gaussianas<a name="id5"></a>